# Clustering Crypto

In [48]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [38]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv('crypto_data.csv')
crypto_df.head(10)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015,2015 coin,X11,True,PoW/PoS,NaN,0
7,BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [39]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df["IsTrading"]== True]
crypto_df = crypto_df.set_index("Unnamed: 0")
crypto_df.index.name = None

In [40]:
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [41]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df = crypto_df[crypto_df["Algorithm"]!= None]
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [42]:
# Remove the "IsTrading" column. 
crypto_df.drop(['IsTrading'], axis=1, inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [43]:
# Remove rows that have at least 1 null value.
crypto_df.dropna(inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [44]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df["TotalCoinsMined"]> 0]
crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [45]:
# Create a new DataFrame that holds only the cryptocurrencies names.
names_df = crypto_df["CoinName"]
names_df.head(10)

42               42 Coin
404              404Coin
1337           EliteCoin
BTC              Bitcoin
ETH             Ethereum
LTC             Litecoin
DASH                Dash
XMR               Monero
ETC     Ethereum Classic
ZEC                ZCash
Name: CoinName, dtype: object

In [46]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(["CoinName"], axis=1, inplace=True)

In [70]:
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [50]:
# Use get_dummies() to create variables for text features.
le = LabelEncoder()
X = crypto_df.copy()
X['Algorithm'] = le.fit_transform(X['Algorithm']) 
X['ProofType'] = le.fit_transform(X['ProofType']) 
X.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,52,15,4.199995e+01,42
404,52,15,1.055185e+09,532000000
1337,66,15,2.927942e+10,314159265359
BTC,47,12,1.792718e+07,21000000
ETH,20,12,1.076842e+08,0


In [51]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:5])

[[ 3.76459118e-01  8.91356555e-01 -1.17108170e-01 -1.52870298e-01]
 [ 3.76459118e-01  8.91356555e-01 -9.39695522e-02 -1.45008997e-01]
 [ 1.21543803e+00  8.91356555e-01  5.24945609e-01  4.48942416e+00]
 [ 7.68237937e-02  1.67233875e-03 -1.16715055e-01 -1.52559984e-01]
 [-1.54120696e+00  1.67233875e-03 -1.14746818e-01 -1.52870298e-01]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [52]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)

In [63]:
# Create a DataFrame with the three principal components.
X_pca=pca.fit_transform(X_scaled)
pcs_df = pd.DataFrame(
   data= X_pca, index=X.index, columns = ["PC 1", "PC 2","PC 3"]) 
pcs_df.head()

,PC 1,PC 2,PC 3
42,-0.417875,0.810296,0.372138
404,-0.396564,0.815135,0.373256
1337,3.124076,2.209780,0.504335
BTC,-0.192083,0.016266,-0.072910
ETH,-0.044116,-1.167492,1.012525


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [64]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)


C:\Users\djkri\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  "KMeans is known to have a memory leak on Windows "


In [65]:
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=4` as it seems to be an elbow point

In [66]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=5)


# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
print(predictions)

[0 0 0 0 3 0 0 3 3 3 2 3 0 2 0 0 0 0 0 0 3 0 0 0 0 0 0 3 0 0 0 0 0 0 2 3 3
 0 0 0 0 0 2 0 0 3 0 0 0 0 2 0 0 0 0 0 0 0 0 3 2 0 0 2 0 0 0 0 0 0 0 0 0 0
 0 0 2 0 0 2 0 2 0 0 0 0 0 0 0 0 0 3 0 0 2 0 0 0 3 3 2 3 0 0 3 0 2 0 0 0 2
 0 0 0 0 0 2 3 0 0 0 0 3 0 0 0 0 0 0 3 0 0 0 2 0 0 2 3 0 3 0 0 0 0 3 0 3 0
 0 0 0 0 0 0 2 0 3 0 0 0 2 0 2 2 0 0 0 0 0 0 0 0 2 0 0 0 2 0 2 3 0 0 3 0 0
 0 0 3 0 0 2 0 0 0 0 2 0 0 3 2 2 0 2 0 3 3 2 0 0 0 0 0 2 0 0 0 2 0 0 0 0 0
 0 0 3 0 2 0 0 2 0 0 3 3 0 0 0 0 0 0 3 2 0 0 2 0 3 0 0 3 2 3 0 3 0 0 2 0 3
 2 0 0 0 0 2 0 0 2 0 2 0 0 2 0 2 3 3 3 3 3 3 0 0 0 0 0 0 0 0 0 0 3 0 0 0 3
 2 0 0 0 0 2 0 0 2 0 0 0 0 0 0 0 3 2 2 0 3 0 0 0 3 2 3 0 0 0 0 3 0 0 0 2 0
 2 0 3 3 3 0 2 0 0 3 0 3 3 3 2 2 0 3 0 0 0 0 3 0 3 2 3 0 0 0 2 0 0 3 2 3 0
 3 0 0 3 0 3 3 2 2 0 0 0 0 3 2 0 0 2 2 3 3 0 3 0 0 0 3 2 3 2 2 0 2 0 0 2 3
 2 0 3 3 2 0 2 3 3 3 3 0 2 3 0 3 0 2 2 2 2 3 0 3 0 0 2 0 2 3 2 0 0 3 3 0 3
 2 0 2 2 0 3 2 0 0 0 0 2 3 2 3 0 0 2 3 0 3 3 3 3 0 0 3 0 3 2 0 0 2 0 0 0 0
 0 0 0 0 2 0 0 0 3 0 3 0 

In [73]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df], axis=1)

#adding names back in
clustered_df['CoinName'] = names_df 

#adding cluster predictions to df
clustered_df['Class'] = predictions

print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.417875,0.810296,0.372138,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.396564,0.815135,0.373256,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,3.124076,2.209780,0.504335,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.192083,0.016266,-0.072910,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.044116,-1.167492,1.012525,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.217954,0.235824,-0.273789,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.482771,1.336609,-0.110253,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.001979,-1.518631,1.334156,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.041907,-1.167016,1.012991,Ethereum Classic,3
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.034542,-1.255502,1.092984,ZCash,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [80]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name="CoinName",
    hover_data=['Algorithm'],
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [75]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(sortable=True, selectable=True)

:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class]

In [76]:
# Print the total number of tradable cryptocurrencies.
index = clustered_df.index
print(f"There are {len(index)} tradable cryptocurrencies.")

There are 532 tradable cryptocurrencies.


In [77]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
mms = MinMaxScaler()
clustered_df_scaled = mms.fit_transform(clustered_df[['TotalCoinSupply','TotalCoinsMined']])
clustered_df_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [78]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df=pd.DataFrame(
data=clustered_df_scaled, columns=['TotalCoinSupply','TotalCoinsMined'], index = clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName'] = clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['Class'] = clustered_df['Class'] 

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,3
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,3
ETC,2.100000e-04,0.000115,Ethereum Classic,3
ZEC,2.100000e-05,0.000007,ZCash,3


In [79]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class", hover_cols=["CoinName"])\

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)